In [8]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

# Where .npz patches are saved
patch_dir = "../json files/patches_5x5"

## I. Patch Extraction Pipeline: VIIRS .npz to DataFrame

This loop processes all `.npz` files for each point-of-interest (POI) and extracts daily patch statistics for each spatial extent (1×1, 3×3, 5×5).

---

### **Algorithm Overview**

1. **Directory Traversal**
    - Iterate over each POI folder in `patch_dir`.
    - For every `.npz` file (corresponding to a date), load patch data.

2. **Patch Extraction for Each Extent**
    - For each extent (`1x1`, `3x3`, `5x5`), extract pixel-level stats for:
        - DNB observed radiance (`DNB_mean`, `DNB_stdDev`)
        - Gap-filled radiance (`Gap_mean`, `Gap_stdDev`)
        - Valid pixel count and patch size (`DNB_valid_px`, `Total_px`)
        - Percent of valid pixels (`Valid_pct`)
        - Quality flags/histograms (`HQ_hist`, `QF_hist`)

3. **Row Construction**
    - Compile all metrics into a dictionary per (POI, date, extent), append to `rows`.

4. **Output**
    - Combine all rows into a tidy DataFrame for downstream analysis and visualization.

---

**This step transforms raw, per-patch .npz data into a flat table of summary statistics, indexed by POI, date, and patch size.**

In [10]:
# Extract center, 3x3, or 5x5 pixels from 5x5 array
def extract_zone(arr, size):
    if arr is None:
        return []
    arr = np.array(arr)
    if arr.ndim != 2 or arr.shape != (5, 5):
        return []
    if size == 1:
        return [arr[2, 2]]
    elif size == 3:
        return arr[1:4, 1:4].flatten().tolist()
    elif size == 5:
        return arr.flatten().tolist()
    return []

# Clean histogram values (cast to int)
def clean_hist(arr):
    return dict(Counter([int(v) for v in arr if v is not None]))

# Compute patch stats
def summarize_patch(values):
    arr = np.array([v for v in values if v is not None])
    valid = arr[arr > 0]
    return {
        'mean': valid.mean() if valid.size else None,
        'std': valid.std() if valid.size else None,
        'valid': int((valid > 0).sum()),
        'total': int(arr.size),
        'pct': float(valid.size / arr.size * 100) if arr.size else None
    }


# Loop through all .npz files
rows = []
for poi in tqdm(os.listdir(patch_dir)):
    poi_path = os.path.join(patch_dir, poi)
    if not os.path.isdir(poi_path):
        continue  # ✅ Skip files like .DS_Store

    for filename in os.listdir(poi_path):
        if not filename.endswith(".npz"):
            continue
        date_str = filename.replace(".npz", "")
        path = os.path.join(poi_path, filename)
        try:
            data = np.load(path, allow_pickle=True)
        except:
            continue

        for size, extent in zip([1, 3, 5], ['1x1', '3x3', '5x5']):
            row = {
                'date': date_str,
                'location': poi,
                'extent': extent,
                'DNB_mean': None,
                'DNB_stdDev': None,
                'Gap_mean': None,
                'Gap_stdDev': None,
                'DNB_valid_px': None,
                'Total_px': None,
                'Valid_pct': None,
                'HQ_hist': None,
                'QF_hist': None
            }

            # DNB
            dnb = extract_zone(data.get('DNB_BRDF_Corrected_NTL'), size)
            if dnb:
                stats = summarize_patch(dnb)
                row.update({
                    'DNB_mean': stats['mean'],
                    'DNB_stdDev': stats['std'],
                    'DNB_valid_px': stats['valid'],
                    'Total_px': stats['total'],
                    'Valid_pct': stats['pct']
                })

            # Gap-filled DNB
            gap = extract_zone(data.get('Gap_Filled_DNB_BRDF_Corrected_NTL'), size)
            if gap:
                stats = summarize_patch(gap)
                row['Gap_mean'] = stats['mean']
                row['Gap_stdDev'] = stats['std']

            # HQ + QF histograms
            hq = extract_zone(data.get('Latest_High_Quality_Retrieval'), size)
            if hq:
                row['HQ_hist'] = clean_hist(hq)

            qf = extract_zone(data.get('QF_Cloud_Mask'), size)
            if qf:
                row['QF_hist'] = clean_hist(qf)

            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

100%|██████████| 11/11 [00:21<00:00,  1.94s/it]


In [11]:
# Fill missing Total_px based on extent
extent_to_total = {'1x1': 1, '3x3': 9, '5x5': 25}
df['Total_px'] = df.apply(
    lambda row: extent_to_total.get(row['extent'], np.nan) if pd.isna(row['Total_px']) else row['Total_px'],
    axis=1
)

# Fill NaN DNB_valid_px with 0
df['DNB_valid_px'] = df['DNB_valid_px'].fillna(0)

# Recompute Valid_pct everywhere: (valid/total)*100, 0 if total is 0 or valid is 0
df['Valid_pct'] = np.where(
    df['Total_px'] > 0,
    df['DNB_valid_px'] / df['Total_px'] * 100,
    0
)

df

,date,location,extent,DNB_mean,DNB_stdDev,Gap_mean,Gap_stdDev,DNB_valid_px,Total_px,Valid_pct,HQ_hist,QF_hist
0,2013-09-02,Mt_Nacolod_Upland,1x1,0.246076,0.000000,0.246076,0.000000,1.0,1.0,100.0,{0: 1},{50: 1}
1,2013-09-02,Mt_Nacolod_Upland,3x3,0.236731,0.051792,0.236731,0.051792,9.0,9.0,100.0,{0: 9},{50: 9}
2,2013-09-02,Mt_Nacolod_Upland,5x5,0.222292,0.049933,0.222292,0.049933,25.0,25.0,100.0,{0: 25},{50: 25}
3,2013-09-16,Mt_Nacolod_Upland,1x1,NaN,NaN,0.108664,0.000000,0.0,1.0,0.0,{3: 1},{738: 1}
4,2013-09-16,Mt_Nacolod_Upland,3x3,NaN,NaN,0.136672,0.019283,0.0,9.0,0.0,"{3: 6, 2: 3}",{738: 9}
...,...,...,...,...,...,...,...,...,...,...,...,...
10945,2013-09-27,Southern_Leyte_Gulf,3x3,NaN,NaN,0.035736,0.019356,0.0,9.0,0.0,{24: 9},"{758: 7, 246: 2}"
10946,2013-09-27,Southern_Leyte_Gulf,5x5,NaN,NaN,0.044017,0.026867,0.0,25.0,0.0,{24: 25},"{758: 18, 246: 7}"
10947,2013-07-22,Southern_Leyte_Gulf,1x1,NaN,NaN,0.010000,0.000000,0.0,1.0,0.0,{4: 1},{758: 1}
10948,2013-07-22,Southern_Leyte_Gulf,3x3,NaN,NaN,0.024385,0.018307,0.0,9.0,0.0,{4: 9},{758: 9}


In [12]:
df[(df["extent"] == "3x3") & (df["location"] == "Tacloban_City")]

,date,location,extent,DNB_mean,DNB_stdDev,Gap_mean,Gap_stdDev,DNB_valid_px,Total_px,Valid_pct,HQ_hist,QF_hist
1096,2013-09-02,Tacloban_City,3x3,23.397640,14.187394,23.397640,14.187394,9.0,9.0,100.000000,{0: 9},"{42: 5, 34: 4}"
1099,2013-09-16,Tacloban_City,3x3,NaN,NaN,35.605633,16.641924,0.0,9.0,0.000000,{3: 9},{746: 9}
1102,2013-07-13,Tacloban_City,3x3,NaN,NaN,39.043689,16.809671,0.0,9.0,0.000000,"{4: 3, 5: 6}",{746: 9}
1105,2013-07-07,Tacloban_City,3x3,NaN,NaN,36.160146,16.479353,0.0,9.0,0.000000,{4: 9},"{746: 5, 738: 4}"
1108,2013-10-23,Tacloban_City,3x3,NaN,NaN,32.833441,17.403205,0.0,9.0,0.000000,{5: 9},"{746: 7, 738: 2}"
...,...,...,...,...,...,...,...,...,...,...,...,...
2176,2013-12-17,Tacloban_City,3x3,4.077320,1.360764,5.616235,4.537842,8.0,9.0,88.888889,"{0: 8, 4: 1}","{58: 5, 122: 1, 50: 3}"
2179,2013-10-06,Tacloban_City,3x3,21.214540,13.861851,29.398954,14.848451,9.0,9.0,100.000000,"{0: 7, 9: 2}",{42: 9}
2182,2013-10-12,Tacloban_City,3x3,NaN,NaN,35.210582,16.530853,0.0,9.0,0.000000,{6: 9},"{746: 2, 234: 5, 226: 1, 738: 1}"
2185,2013-09-27,Tacloban_City,3x3,23.168039,11.159414,31.448856,12.984177,9.0,9.0,100.000000,"{0: 7, 14: 2}",{42: 9}


## II. Interactive NTL Patch Time Series Dashboard (Plotly)

This dashboard visualizes VIIRS nighttime lights (NTL) statistics for multiple locations and patch sizes, enabling rapid comparison of radiance, coverage, and quality over time and space.

---

### **How it works**

- **Patch Size & Location**  
  User can select any point of interest (POI) and patch extent (`1x1`, `3x3`, `5x5`). Patch size determines the spatial footprint (1, 9, or 25 pixels per patch).

- **Layout**  
  - **Top panel:** Bar chart of valid pixel coverage (%) per day (proportion of fresh, HQ=0 pixels in the patch).
  - **Bottom panel:**  
    - Red line: Gap-filled NTL mean (±1 stddev shaded band).  
    - Blue dots: Observed NTL mean (±1 stddev error bars for per-day uncertainty).

- **Dropdowns/Buttons**  
  - Select different locations and patch sizes interactively.  
  - Default view: `3x3` patch at Tacloban_City.

- **Landfall Annotation**  
  A vertical line/annotation marks Typhoon Haiyan’s landfall for reference.

- **Visibility Control**  
  Only traces for the currently selected location and patch size are shown; all others are hidden.

---

### **Usage Notes**
- **Valid pixel %** reflects the average daily percent of pixels in the patch with high-quality (HQ=0) values.
- **Gap-filled radiance** represents interpolated or “filled” NTL for cloudy or missing days.
- **Observed radiance** is shown only when direct measurements are available.

This tool provides a compact, interactive way to analyze NTL trends, cloud coverage, and recovery at multiple scales after a disaster or for monitoring urban dynamics.

In [22]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# --- Setup ---
extent_codes = ['1x1', '3x3', '5x5']
extent_label_map = {'1x1': '1 px radius', '3x3': '3 px radius', '5x5': '5 px radius'}

poi_data = [
    {'name': 'Tacloban_City'}, {'name': 'Ormoc_City'}, {'name': 'Baybay_City'},
    {'name': 'Palo'}, {'name': 'Alang_Alang'}, {'name': 'Guiuan'},
    {'name': 'Samar_Forest_Reserve'}, {'name': 'Mt_Nacolod_Upland'},
    {'name': 'Central_Leyte_Forest'}, {'name': 'Southern_Leyte_Gulf'}
]
locations = [p['name'] for p in poi_data]

fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.20, 0.70],
    shared_xaxes=True,
    vertical_spacing=0.10,
    subplot_titles=("Observed Pixel Coverage (%)", "NTL Radiance Time Series")
)

trace_settings = []

for extent in extent_codes:
    for loc in locations:
        dfl = df[(df['extent'] == extent) & (df['location'] == loc)].copy()
        if dfl.empty:
            continue
        # --- Always make sure dates are sorted and correct type
        dfl['date'] = pd.to_datetime(dfl['date'])
        dfl = dfl.sort_values('date')

        # Coverage bar
        fig.add_trace(
            go.Bar(
                x=dfl['date'], y=dfl['Valid_pct'],
                name='Coverage', marker_color='green', opacity=1, showlegend=False
            ), row=1, col=1
        )
        trace_settings.append((extent, loc, 'coverage'))

        # Gap-Filled mean and stddev band
        dfl_gap = dfl.dropna(subset=['Gap_mean', 'Gap_stdDev'])
        gap_upper = dfl_gap['Gap_mean'] + dfl_gap['Gap_stdDev']
        gap_lower = dfl_gap['Gap_mean'] - dfl_gap['Gap_stdDev']

        # Gap mean (red line)
        fig.add_trace(
            go.Scatter(
                x=dfl_gap['date'], y=dfl_gap['Gap_mean'],
                mode='lines',
                line=dict(color='red'),
                name='Gap-Filled',
                showlegend=True if (extent == '3x3' and loc == locations[0]) else False
            ), row=2, col=1
        )
        trace_settings.append((extent, loc, 'gap'))

        # Stddev band (upper, invisible)
        fig.add_trace(
            go.Scatter(
                x=dfl_gap['date'], y=gap_upper,
                mode='lines', line=dict(width=0),
                showlegend=False, hoverinfo='skip'
            ), row=2, col=1
        )
        trace_settings.append((extent, loc, 'gap_std_upper'))

        # Stddev band (lower, fill)
        fig.add_trace(
            go.Scatter(
                x=dfl_gap['date'], y=gap_lower,
                mode='lines', line=dict(width=0),
                fill='tonexty', fillcolor='rgba(255,0,0,0.13)',
                showlegend=False, hoverinfo='skip'
            ), row=2, col=1
        )
        trace_settings.append((extent, loc, 'gap_std_lower'))

        # Observed DNB (blue dots)
        dfl_dnb = dfl.dropna(subset=['DNB_mean', 'DNB_stdDev'])
        fig.add_trace(
            go.Scatter(
                x=dfl_dnb['date'],
                y=dfl_dnb['DNB_mean'],
                mode='markers',
                marker=dict(size=6, color='blue'),
                name='Observed',
                error_y=dict(
                    type='data',
                    array=dfl_dnb['DNB_stdDev'],
                    visible=True,
                    color='blue',
                    thickness=0.5,
                    width=2  # width of error bar cap
                ),
                showlegend=True,
            ),
            row=2, col=1
        )
        trace_settings.append((extent, loc, 'dnb'))

# --- Helper for visibility control ---
def vis_array(sel_extent, sel_loc):
    return [
        (extent == sel_extent and loc == sel_loc)
        for extent, loc, _ in trace_settings
    ]

# --- Dropdowns/buttons for location and extent ---
location_buttons = [
    dict(
        label=loc, method='update',
        args=[
            {'visible': vis_array('3x3', loc)},
            {'title': f'NTL Radiance – 3 px radius – {loc}'}
        ]
    )
    for loc in locations
]
extent_buttons = [
    dict(
        label=extent_label_map[extent], method='update',
        args=[
            {'visible': vis_array(extent, locations[0])},
            {'title': f'NTL Radiance – {extent_label_map[extent]} – {locations[0]}'}
        ]
    )
    for extent in extent_codes
]

# --- Set initial visibility: 3x3, Tacloban by default
initial_visible = vis_array('3x3', locations[0])
for i, vis in enumerate(initial_visible):
    fig.data[i].visible = vis

# --- Haiyan Landfall line/annotation
fig.add_vline(
    x=pd.to_datetime("2013-11-08"),
    line_width=2, line_dash="dash", line_color="blue"
)
fig.add_annotation(
    x="2013-11-08", y=1.02, yref='paper',
    text="Haiyan Landfall", showarrow=False,
    font=dict(color="black"), bgcolor="white",
    bordercolor="black", borderwidth=1, xanchor="left"
)

# --- Layout ---
fig.update_layout(
    updatemenus=[
        dict(type="dropdown", direction="down", buttons=location_buttons,
             showactive=True, x=1.1, xanchor="center", y=0.6, yanchor="bottom"),
        dict(type="buttons", direction="right", buttons=extent_buttons,
             showactive=True, x=1.0, y=1.15, xanchor="right", yanchor="top")
    ],
    height=700, width=1200, template='plotly_white',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis2=dict(title='Date', rangeslider=dict(visible=True)),
    yaxis=dict(title='Px Coverage (%)', range=[0, 100]),
    yaxis2=dict(title='Radiance (nW·cm⁻²·sr⁻¹)')#, range=[0, 2])
)

fig.show()

## III. Summary Table Algorithm for VIIRS Patch Time Series

This summary statistics process generates key indicators for each point of interest (POI) and patch extent (1×1, 3×3, 5×5 pixels) using VIIRS nighttime lights data. The design ensures valid comparison across patch sizes and robustly incorporates data quality flags.

---

### **Key Steps**

1. **Patch Extent Definition**
    - `1x1` patch: 1 pixel per day
    - `3x3` patch: 9 pixels per day
    - `5x5` patch: 25 pixels per day

2. **Valid and Gap-Filled Pixel Counting**
    - For each POI and extent, count the total number of valid (fresh, HQ=0) and gap-filled pixels across all days.
    - For `1x1`, this is simply the number of days the pixel is valid or gap-filled.
    - For `3x3` and `5x5`, sum valid/gap-filled pixels across all days (e.g., Valid Pixels = sum of `DNB_valid_px`).

3. **Percent Calculation**
    - **Valid Pixels (%)**:  
      ```
      Valid Pixels (%) = (Total Valid Pixels) / (Number of Days × Pixels per Patch) × 100
      ```
    - **Gap-Filled Pixels (%)**:  
      ```
      Gap-Filled Pixels (%) = 100 - Valid Pixels (%)
      ```
    - Both the count and percent are shown for interpretability (e.g., "115 (68.4%)").

4. **Quality Flags from `HQ_hist`**
    - **% HQ = 0 (Same-day)**:  
      Proportion of all pixels (days × patch size) classified as "fresh" (HQ=0), based on `HQ_hist`.
    - **Median Latest HQ Retrieval (days)**:  
      Median value from all `HQ_hist` retrievals, showing the typical "age" of data used for each pixel.

5. **Mean Radiance and Dropout**
    - **DNB Mean** and **Gap-Filled Mean ± STD** are reported for each patch/POI.
    - **Max Dropout (days)**:  
      The longest streak of consecutive days with zero valid pixels, indicating worst-case observation gap.

6. **Consistency for 1x1**
    - For `1x1`, **Valid Pixels (%)** and **% HQ = 0 (Same-day)** are both calculated from `HQ_hist`, so they always match and represent the number of days the pixel is fresh.

---

### **Table Output Example**

| Location         | Valid Pixels (%) | Gap-Filled Pixels (%) | DNB Mean | Gap-Filled Mean ± STD | Max Dropout (days) | Median Latest HQ Retrieval (days) | % HQ = 0 (Same-day) |
|------------------|-----------------|----------------------|----------|----------------------|--------------------|------------------------------|----------------------|
| Tacloban_City    | 119 (32.6%)     | 246 (67.4%)          | 25.19    | 29.43 ± 0.00         | 11                 | 2                            | 25.82                |
| ...              | ...             | ...                  | ...      | ...                  | ...                | ...                          | ...                  |

---

**Notes:**
- For 3x3 and 5x5 patches, percentages represent total pixel coverage across the time period.
- For 1x1, "pixels" = "days."
- All calculations are robust to missing values.

In [18]:
from ast import literal_eval

def parse_hist(series):
    merged = {}
    all_vals = []
    for val in series.dropna():
        h = literal_eval(val) if isinstance(val, str) else val
        all_vals.extend([int(k)] * int(v) for k, v in h.items())
        for k, v in h.items():
            k = int(k)
            merged[k] = merged.get(k, 0) + float(v)
    total = sum(merged.values()) or 1
    return {
        'rel': {k: v / total for k, v in merged.items()},
        'raw': merged,
        'vals': [k for sublist in all_vals for k in sublist]
    }

def max_dropout(valid_flags):
    max_run = run = 0
    for v in valid_flags:
        if v == 0:
            run += 1
            max_run = max(max_run, run)
        else:
            run = 0
    return max_run

summary_dict = {ec: [] for ec in extent_codes}
poi_locations = [poi['name'] for poi in poi_data]

extent_to_total = {'1x1': 1, '3x3': 9, '5x5': 25}

for extent_code in extent_codes:
    df_e = df[df['extent'] == extent_code].copy()
    n_pix = extent_to_total[extent_code]
    for loc in poi_locations:
        dfl = df_e[df_e['location'] == loc].copy()
        if dfl.empty:
            continue

        row = {'Location': loc}

        if extent_code == '1x1':
            total_days = len(dfl)
            # Use HQ_hist as the definitive source for "fresh" pixels
            if 'HQ_hist' in dfl.columns:
                hq = parse_hist(dfl['HQ_hist'])
                fresh_days = int(hq['raw'].get(0, 0))
                hq_vals = hq['vals']
            else:
                fresh_days = dfl['DNB_mean'].notna().sum()
                hq_vals = []
            valid_pct = (fresh_days / total_days * 100) if total_days else 0
            gapfilled_days = total_days - fresh_days
            gapfilled_pct = 100 - valid_pct

            dnb_mean = dfl['DNB_mean'].mean()
            gap_mean = dfl['Gap_mean'].mean()
            gap_std = dfl['Gap_stdDev'].mean()

            row.update({
                'Valid Pixels (%)': f"{fresh_days} ({valid_pct:.1f}%)",
                'Gap-Filled Pixels (%)': f"{gapfilled_days} ({gapfilled_pct:.1f}%)",
                'DNB Mean': f"{dnb_mean:.2f}" if not np.isnan(dnb_mean) else "—",
                'Gap-Filled Mean ± STD': f"{gap_mean:.2f} ± {gap_std:.2f}" if not np.isnan(gap_mean) else "—",
                'Max Dropout (days)': max_dropout((dfl['DNB_mean'].notna()).astype(int)),
                'Median Latest HQ Retrieval (days)': int(np.median(hq_vals)) if hq_vals else '—',
                '% HQ = 0 (Same-day)': round(valid_pct, 2)
            })

        else:
            total_px = len(dfl) * n_pix
            valid_pix = dfl['DNB_valid_px'].fillna(0).sum()
            gapfilled_pix = total_px - valid_pix

            valid_pct = (valid_pix / total_px * 100) if total_px > 0 else 0
            gapfilled_pct = (gapfilled_pix / total_px * 100) if total_px > 0 else 0

            dnb_mean = dfl['DNB_mean'].mean()
            dnb_std = dfl['DNB_stdDev'].mean()
            gap_mean = dfl['Gap_mean'].mean()
            gap_std = dfl['Gap_stdDev'].mean()

            if 'HQ_hist' in dfl.columns:
                hq = parse_hist(dfl['HQ_hist'])
                hq_vals = hq['vals']
                median_hq = int(np.median(hq_vals)) if hq_vals else '—'
                pct_hq0 = round(hq['rel'].get(0, 0) * 100, 2)
            else:
                median_hq = '—'
                pct_hq0 = '—'

            row.update({
                'Valid Pixels (%)': f"{int(valid_pix)} ({valid_pct:.1f}%)",
                'Gap-Filled Pixels (%)': f"{int(gapfilled_pix)} ({gapfilled_pct:.1f}%)",
                'DNB Mean ± STD': f"{dnb_mean:.2f} ± {dnb_std:.2f}" if not np.isnan(dnb_mean) else "—",
                'Gap-Filled Mean ± STD': f"{gap_mean:.2f} ± {gap_std:.2f}" if not np.isnan(gap_mean) else "—",
                'Max Dropout (days)': max_dropout((dfl['DNB_valid_px'].fillna(0) > 0).astype(int)),
                'Median Latest HQ Retrieval (days)': median_hq,
                '% HQ = 0 (Same-day)': pct_hq0
            })

        summary_dict[extent_code].append(row)

summary_columns_1x1 = [
    'Location', 'Valid Pixels (%)', 'Gap-Filled Pixels (%)', 'DNB Mean',
    'Gap-Filled Mean ± STD', 'Max Dropout (days)',
    'Median Latest HQ Retrieval (days)', '% HQ = 0 (Same-day)'
]
summary_columns_patch = [
    'Location', 'Valid Pixels (%)', 'Gap-Filled Pixels (%)', 'DNB Mean ± STD',
    'Gap-Filled Mean ± STD', 'Max Dropout (days)',
    'Median Latest HQ Retrieval (days)', '% HQ = 0 (Same-day)'
]

summary_1x1 = pd.DataFrame(summary_dict['1x1'])[summary_columns_1x1]
summary_3x3 = pd.DataFrame(summary_dict['3x3'])[summary_columns_patch]
summary_5x5 = pd.DataFrame(summary_dict['5x5'])[summary_columns_patch]

In [19]:
summary_1x1

,Location,Valid Pixels (%),Gap-Filled Pixels (%),DNB Mean,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Tacloban_City,94 (25.8%),271 (74.2%),25.19,29.43 ± 0.00,11,2,25.75
1,Ormoc_City,97 (26.6%),268 (73.4%),16.06,21.11 ± 0.00,15,2,26.58
2,Baybay_City,78 (21.4%),287 (78.6%),6.06,9.46 ± 0.00,15,3,21.37
3,Palo,104 (28.5%),261 (71.5%),7.42,7.62 ± 0.00,13,2,28.49
4,Alang_Alang,119 (32.6%),246 (67.4%),1.89,2.16 ± 0.00,11,2,32.60
5,Guiuan,76 (20.8%),289 (79.2%),4.45,5.96 ± 0.00,15,2,20.82
6,Samar_Forest_Reserve,130 (35.6%),235 (64.4%),0.51,0.17 ± 0.00,12,1,35.62
7,Mt_Nacolod_Upland,130 (35.6%),235 (64.4%),0.48,0.19 ± 0.00,16,1,35.62
8,Central_Leyte_Forest,80 (21.9%),285 (78.1%),1.17,0.16 ± 0.00,18,3,21.92
9,Southern_Leyte_Gulf,55 (15.1%),310 (84.9%),0.39,0.06 ± 0.00,61,4,15.07


In [20]:
summary_3x3

,Location,Valid Pixels (%),Gap-Filled Pixels (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Tacloban_City,1013 (30.8%),2272 (69.2%),20.89 ± 9.01,28.43 ± 14.40,11,2,24.73
1,Ormoc_City,966 (29.4%),2319 (70.6%),11.64 ± 6.11,14.07 ± 7.24,15,2,23.26
2,Baybay_City,870 (26.5%),2415 (73.5%),4.41 ± 2.54,6.61 ± 3.27,15,3,21.66
3,Palo,1107 (33.7%),2178 (66.3%),4.81 ± 3.22,6.01 ± 3.58,13,2,27.65
4,Alang_Alang,1105 (33.6%),2180 (66.4%),1.08 ± 0.82,0.84 ± 1.03,11,2,33.03
5,Guiuan,931 (28.3%),2354 (71.7%),3.16 ± 1.62,3.49 ± 2.01,15,2,22.07
6,Samar_Forest_Reserve,1105 (33.6%),2180 (66.4%),0.52 ± 0.26,0.16 ± 0.06,12,1,35.65
7,Mt_Nacolod_Upland,1076 (32.8%),2209 (67.2%),0.54 ± 0.27,0.19 ± 0.07,13,1,35.10
8,Central_Leyte_Forest,716 (21.8%),2569 (78.2%),1.09 ± 0.35,0.17 ± 0.07,18,3,23.10
9,Southern_Leyte_Gulf,376 (11.4%),2909 (88.6%),0.37 ± 0.10,0.05 ± 0.03,61,4,15.94


In [21]:
summary_5x5

,Location,Valid Pixels (%),Gap-Filled Pixels (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Tacloban_City,2616 (28.7%),6509 (71.3%),12.85 ± 10.36,15.93 ± 14.42,11,2,24.47
1,Ormoc_City,2466 (27.0%),6659 (73.0%),7.40 ± 6.30,8.22 ± 7.51,15,2,23.29
2,Baybay_City,2173 (23.8%),6952 (76.2%),2.69 ± 2.41,3.24 ± 3.49,15,3,22.37
3,Palo,2995 (32.8%),6130 (67.2%),3.29 ± 3.06,3.80 ± 3.68,13,2,28.96
4,Alang_Alang,2986 (32.7%),6139 (67.3%),0.77 ± 0.70,0.52 ± 0.70,11,1,34.12
5,Guiuan,2275 (24.9%),6850 (75.1%),2.00 ± 1.74,1.70 ± 2.00,15,2,23.08
6,Samar_Forest_Reserve,3012 (33.0%),6113 (67.0%),0.53 ± 0.31,0.17 ± 0.08,12,1,35.40
7,Mt_Nacolod_Upland,2920 (32.0%),6205 (68.0%),0.57 ± 0.39,0.19 ± 0.09,13,1,34.56
8,Central_Leyte_Forest,1923 (21.1%),7202 (78.9%),1.11 ± 0.49,0.16 ± 0.08,18,3,22.99
9,Southern_Leyte_Gulf,991 (10.9%),8134 (89.1%),0.36 ± 0.10,0.05 ± 0.03,61,4,15.86
